<a href="https://colab.research.google.com/github/deejay14/Jhakas-NewsWala/blob/main/LSH_Implementation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
import numpy as np
import pandas as pd
import re, nltk, spacy, gensim
# Sklearn
from sklearn.decomposition import LatentDirichletAllocation, TruncatedSVD
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import GridSearchCV
from pprint import pprint
# Plotting tools
import matplotlib.pyplot as plt
%matplotlib inline
#LSH
import re                                                                       # usual lemmetisation
from pprint import pprint
import nltk, spacy, gensim
from sklearn.feature_extraction.text import CountVectorizer

In [6]:
 pip install dateparser

     |████████████████████████████████| 286kB 5.8MB/s 


In [7]:
url='https://raw.githubusercontent.com/deejay14/Jhakas-NewsWala/main/final_fixed_scrapped.csv'    #reads final document
df = pd.read_csv(url) 

In [8]:
df

,Unnamed: 0,Title,Article,totalwords
0,0,Rs 400-cr ‘bungling’ by Amar Singh: HC seeks E...,The Allahabad High Court has directed the Enfo...,341
1,1,J&K HC orders FIR against teacher who fails to...,The J&K High Court on Friday asked a teacher t...,335
2,2,Crime in Cyberia: an incomplete list of offenders,An unknown Indian hacker has been charged with...,382
3,3,450 cannabis plants found on govt land in Mundhwa,The interrogation of two marijuana peddlers ab...,372
4,4,"Nagaland: 9 bullet-riddled bodies found,Karbi-...",Nine highly decomposed bodies were found stack...,449
...,...,...,...,...
7838,7838,"13 dead, one million evacuated in China after ...",At least 18 people were killed due to a landsl...,397
7839,7839,"Book Karnataka, Maharashtra CMs for culpable h...",With the flood situation in the state continui...,500
7840,7840,Romanian researcher develops disc-shaped flyin...,"Razan Sabie, an aerodynamicist who operates fr...",346
7841,7841,Is asteroid Apophis capable of triggering a ts...,It was around a few months back that Iain McDo...,412


In [9]:
df['Article']=df['Article'].apply(str)
data_list = df['Article'].values.tolist()

df['Title']=df['Title'].apply(str)
data_titles = df['Title'].values.tolist()

data_list[:1]

['The Allahabad High Court has directed the Enforcement Directorate (ED) to file a status report, along with an affidavit, in connection with the investigation carried out in connection with the allegations of financial misappropriation to the tune over Rs 400 crore by former Samajwadi Party leader Amar Singh. The court has fixed October 8 for next hearing.A Division Bench of Chief Justice D Y Chandrachud and Justice Yashwant Varma passed the order on Thursday, while hearing a PIL filed by Ashok Kumar Singh, a resident of Kanpur, through his counsel Ashok Pandey.The petitioner, who was at one point district chief of SP in Kanpur, had pointed out that a three-judge bench had, in its order on August 1, 2013, directed further investigation by the ED in connection with the allegations against Amar Singh. It was alleged that as the head of the UP development council, he had misappropriated funds to the amount of Rs 400 crore. An FIR in this regard was registered in Kanpur on October 15, 200

Data Cleaning

In [10]:


def get_lemmatized_clean_data(df):
    # Convert to list
    data = df['Article'].values.tolist()

    # Remove Emails
    data = [re.sub('\S*@\S*\s?', '', sent) for sent in data]

    # Remove new line characters
    data = [re.sub('\s+', ' ', sent) for sent in data]

    # Remove distracting single quotes
    data = [re.sub("\'", "", sent) for sent in data]

    pprint(data[:1])

    def sent_to_words(sentences):
        for sentence in sentences:
            yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

    data_words = list(sent_to_words(data))

    print(data_words[:1])
    
    def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
        """https://spacy.io/api/annotation"""
        texts_out = []
        for sent in texts:
            doc = nlp(" ".join(sent)) 
            texts_out.append(" ".join([token.lemma_ if token.lemma_ not in ['-PRON-'] else '' for token in doc if token.pos_ in allowed_postags]))
        return texts_out

    # Initialize spacy 'en' model, keeping only tagger component (for efficiency)
    # Run in terminal: python3 -m spacy download en
    nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])

    # Do lemmatization keeping only Noun, Adj, Verb, Adverb
    data_lemmatized = lemmatization(data_words, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

    return data_lemmatized

In [11]:
data_lemmatized = get_lemmatized_clean_data(df)

['The Allahabad High Court has directed the Enforcement Directorate (ED) to '
 'file a status report, along with an affidavit, in connection with the '
 'investigation carried out in connection with the allegations of financial '
 'misappropriation to the tune over Rs 400 crore by former Samajwadi Party '
 'leader Amar Singh. The court has fixed October 8 for next hearing.A Division '
 'Bench of Chief Justice D Y Chandrachud and Justice Yashwant Varma passed the '
 'order on Thursday, while hearing a PIL filed by Ashok Kumar Singh, a '
 'resident of Kanpur, through his counsel Ashok Pandey.The petitioner, who was '
 'at one point district chief of SP in Kanpur, had pointed out that a '
 'three-judge bench had, in its order on August 1, 2013, directed further '
 'investigation by the ED in connection with the allegations against Amar '
 'Singh. It was alleged that as the head of the UP development council, he had '
 'misappropriated funds to the amount of Rs 400 crore. An FIR in this re

In [12]:
len(data_lemmatized)

7843

Tokenizer:

In [13]:
def tokenizer(sentences):
  def sent_to_words(sentences):
    for sentence in sentences:
      yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations
  tokens=list(sent_to_words(sentences))
  return tokens


In [14]:
check=tokenizer(data_lemmatized)
len(check)

7843

In [15]:
pip install datasketch

     |████████████████████████████████| 71kB 3.0MB/s 


Minhash implementation : Creation of Forest (equivalent to training in LDA)

In [16]:
import time
from datasketch import MinHash, MinHashLSHForest

permutations = 128

#Number of Recommendations to return
num_recommendations = 1
def get_forest(perms):
    start_time = time.time()
    
    minhash = []
    
    for tok in check:
      m = MinHash(num_perm=perms)
      for s in tok:
          m.update(s.encode('utf8'))
      minhash.append(m)
        
    forest = MinHashLSHForest(num_perm=perms)
    
    for i,m in enumerate(minhash):
        forest.add(i,m)
        
    forest.index()
    
    print('It took %s seconds to build forest.' %(time.time()-start_time))
    
    return forest

Generating query in the next two cells




In [17]:
art_id =5                              # enter article id to be searched here
def preprocess(text):
    text = re.sub(r'[^\w\s]','',text)
    tokens = text.lower()
    tokens = tokens.split()
    return tokens

texta=preprocess(data_lemmatized[art_id])



In [18]:
print(texta)

['today', 'election', 'hold', 'opposition', 'boycott', 'shubhajit', 'find', 'country', 'seek', 'retain', 'identity', 'grow', 'islamic', 'influence', 'unfinished', 'business', 'mamoon', 'worried', 'man', 'today', 'year', 'old', 'history', 'professor', 'warn', 'child', 'would', 'snap', 'relation', 'follow', 'extremist', 'ideology', 'spread', 'political', 'force', 'country', 'bangladesh', 'head', 'poll', 'today', 'elect', 'th', 'parliament', 'mamoon', 'many', 'other', 'citizen', 'country', 'election', 'usual', 'issue', 'corruption', 'economy', 'law', 'order', 'election', 'centre', 'very', 'idea', 'bangladesh', 'should', 'secular', 'democracy', 'theocratic', 'state', 'islamic', 'nation', 'protest', 'call', 'opposition', 'bnp', 'align', 'islamist', 'more', 'people', 'kill', 'clash', 'past', 'half', 'month', 'bnp', 'boycott', 'poll', 'call', 'farce', 'ask', 'people', 'stay', 'away', 'statement', 'when', 'call', 'hour', 'begin', 'morning', 'include', 'polling', 'day', 'bnp', 'say', 'one', 'ho

**Prediction Process :**

In [19]:
def predict(text, df, perms, num_results, forest):
    start_time = time.time()
    
    tokens = text
    m = MinHash(num_perm=perms)
    for s in tokens:
        m.update(s.encode('utf8'))
        
    idx_array = np.array(forest.query(m, num_results))
    if len(idx_array) == 0:
        return None # if your query is empty, return none
    
    result = df.iloc[idx_array]['Article']
    
    print('It took %s seconds to query forest.' %(time.time()-start_time))
    
    return result

In [20]:
forest = get_forest(permutations)

It took 35.790191411972046 seconds to build forest.


In [21]:
num_recommendations = 10
result = predict(texta, df, permutations, num_recommendations, forest)
print('\n Top Recommendation(s) is(are) \n', result)


It took 0.026519060134887695 seconds to query forest.

 Top Recommendation(s) is(are) 
 4224    About this series — \nAh, 2020: The year when ...
5       With todays elections being held under an Opp...
6       With todays elections being held under an Opp...
1030    Retd Delhi HC Chief Justice and the man behind...
1095    In a curious submission,the government on Frid...
2322    The Goa Police has sought additional security ...
2007    Twenty-seven South Asian workers,including 24 ...
4154    Read more from our '2020, the year of...' seri...
604     Fair,transparent court martial proceedings wou...
4478    By Meredith Jones\nKim Kardashian West has ann...
Name: Article, dtype: object


In [24]:
final_recom =list(result.index)

In [25]:
print(final_recom)

[4224, 5, 6, 1030, 1095, 2322, 2007, 4154, 604, 4478]
